## Calibration tests
### Pose2Sim calibration.py functions
### Data test trampo (CRME)

Auteurs : Léa Drolet-Roy

Création : 2025-03-17

Dernière modification :

### Imports

In [2]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import glob

import pyswarms as ps
from pyswarms.utils.plotters import plot_cost_history

from Calibration import calibration as cali

np.set_printoptions(precision=4, suppress=True)

### Global parameters

In [2]:
CHECKERBOARD = (9,4)
image_size = (1080,1920)

aruco_name = cv2.aruco.DICT_4X4_100
aruco_dict = cv2.aruco.getPredefinedDictionary(aruco_name)
charuco_board = cv2.aruco.CharucoBoard(CHECKERBOARD, 112, 86, aruco_dict)

cams = ['c1', 'c2', 'c3', 'c5', 'c6', 'c7', 'c8']
time_threshold = 15

path = r'C:\Users\LEA\Desktop\Poly\Trampo\video_test'
calib_dir = r'C:\Users\LEA\Desktop\Poly\Trampo\video_test\corners_found'

### Test Charuco Board creation

In [ ]:
LENGTH_PX = 640
MARGIN_PX = 20

def create_and_save_new_board():
    dictionary = cv2.aruco.getPredefinedDictionary(aruco_name)
    board = cv2.aruco.CharucoBoard(CHECKERBOARD, 112, 86, dictionary)
    size_ratio = CHECKERBOARD[1] / CHECKERBOARD[0]
    img = cv2.aruco.CharucoBoard.generateImage(board, (LENGTH_PX, int(LENGTH_PX*size_ratio)), marginSize=MARGIN_PX)
    cv2.imshow("img", img)
    cv2.waitKey(2000)

create_and_save_new_board()

: 

In [3]:
def findCorners(img=None, im_name=None):
    if im_name is not None:
        img = cv2.imread(im_name)
    
    if img is None:
        raise ValueError('No image nor image path specified')
    
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    params = cv2.aruco.DetectorParameters()
    corners, ids, _ = cv2.aruco.detectMarkers(gray, aruco_dict, parameters=params)

    if ids is not None: # and len(ids) >= 4:
        _, charuco_corners, charuco_ids = cv2.aruco.interpolateCornersCharuco(corners, ids, gray, charuco_board)
        if charuco_corners is not None and len(charuco_corners) > 0:
            return corners, charuco_corners, charuco_ids
        
    return None, None, None

### Extract and save frames where CharucoBoard is detected

In [ ]:
# Set paths
output_dir = os.path.join(path, 'corners_found')
os.makedirs(output_dir, exist_ok=True)  # Ensure output folder exists

intrinsics_extension = 'mp4'
intrinsics_cam_listdirs_names = next(os.walk(os.path.join(path, 'intrinsics')))[1]

detector_params = cv2.aruco.DetectorParameters()

for i, cam in enumerate(intrinsics_cam_listdirs_names):
    os.makedirs(os.path.join(output_dir, cam), exist_ok=True)
    video_path = glob.glob(os.path.join(path, 'intrinsics', cam, f'*.{intrinsics_extension}'))
    if len(video_path) == 0:
        raise ValueError(f'The folder {os.path.join(path, 'intrinsics', cam)} does not contain any .{intrinsics_extension} video files.')

    frame_count = 0
    valid_frame_count = 0

    for img_path in video_path:
        cap = cv2.VideoCapture(img_path)
        frame_count = 0
        valid_frame_count = 0

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            corners, charuco_corners, charuco_ids = findCorners(frame)

            if charuco_corners is not None and charuco_ids is not None:
                valid_frame_count += 1
                frame_filename = os.path.join(output_dir, cam, f"charuco_frame_{valid_frame_count:03d}.png")
                cv2.imwrite(frame_filename, frame)

            frame_count += 1

cv2.destroyAllWindows()

print(f"Processed {frame_count} frames, saved {valid_frame_count} valid Charuco frames.")

Processed 2654 frames, saved 836 valid Charuco frames.


In [4]:
def calibrate_intrinsics(calib_dir, cams, image_size, camera_matrix=None, dist_coeffs=None):
    intrinsics_extension = 'png'
    ret, C, S, D, K = [], [], [], [], []

    for cam in cams:
        image_files = sorted(glob.glob(os.path.join(calib_dir, cam, "*.png")))  # Adjust extension if needed
        if len(image_files) == 0:
            raise ValueError(f'The folder {os.path.join(calib_dir, cam)} does not contain images with .{intrinsics_extension}')
        
        # Data storage
        imgpoints = []  # Detected charuco corners
        charuco_ids = []  # Corresponding IDs
        image_size = None  # To store image resolution

        # Process each image
        for img_path in image_files:
            img = cv2.imread(img_path)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            if image_size is None:
                image_size = gray.shape[::-1]  # Get image size from first frame

            # Detect Aruco markers
            corners, ids, _ = cv2.aruco.detectMarkers(gray, aruco_dict)

            if ids is not None:
                # Refine detection by interpolating Charuco corners
                _, charuco_corners, charuco_ids_found = cv2.aruco.interpolateCornersCharuco(
                    markerCorners=corners, markerIds=ids, image=gray, board=charuco_board)

                if charuco_corners is not None and len(charuco_corners) > 6:  # Ensure enough points
                    imgpoints.append(charuco_corners)
                    charuco_ids.append(charuco_ids_found)
        
        # Ensure we have enough valid frames
        if len(imgpoints) < 10:
            raise ValueError(f"Not enough valid images for calibration of {cam}! Need at least 10.")

        # Convert to NumPy arrays
        imgpoints = [np.array(p, dtype=np.float32) for p in imgpoints]
        charuco_ids = [np.array(p, dtype=np.int32) for p in charuco_ids]

        # Run calibration
        ret_cam, mtx, dist, _, _ = cv2.aruco.calibrateCameraCharuco(
            charucoCorners=imgpoints,
            charucoIds=charuco_ids,
            board=charuco_board,
            imageSize=image_size,
            cameraMatrix=camera_matrix,  # Use initialized matrix
            distCoeffs=dist_coeffs,
            flags=cv2.CALIB_USE_INTRINSIC_GUESS+cv2.CALIB_USE_LU
        )

        # Save results
        h, w = map(np.float32, image_size)
        ret.append(ret_cam)
        C.append(cam)
        S.append([w, h])
        D.append(dist)
        K.append(mtx)

        # Print results
        print(f"\nCalibration completed for camera: {cam}")
        print(f'Error: {ret_cam:.4f}')
        print("Camera Matrix:\n", mtx)
        print("Distortion Coefficients:\n", dist)

    return ret, C, S, D, K


In [ ]:
# Try setting an initial camera matrix
h, w = image_size
focal_length = 2000  # Approximate focal length
camera_matrix_init = np.array([
    [focal_length, 0, w / 2],
    [0, focal_length, h / 2],
    [0, 0, 1]], dtype=np.float32)

ret, C, S, D, K = calibrate_intrinsics(calib_dir, cams, image_size, camera_matrix_init)

In [4]:
K = np.array(K)
np.savez('Intrinsics_K_trampo.npz', K)

D = np.array(D)
np.savez('Intrinsics_D_trampo.npz', D)

In [4]:
K = np.load('Intrinsics_K_trampo.npz')['arr_0']
D = np.load('Intrinsics_D_trampo.npz')['arr_0']

print(f'Intrinsics: \n{K[0]} \n{K[1]} \n{K[2]} \n{K[3]} \n{K[4]} \n{K[5]}')

Intrinsics: 
[[1249.0508    0.      936.0954]
 [   0.     1247.4435  536.2969]
 [   0.        0.        1.    ]] 
[[1251.7008    0.      961.0123]
 [   0.     1240.7726  545.1083]
 [   0.        0.        1.    ]] 
[[2158.3037    0.      968.1835]
 [   0.     2163.0135  538.9601]
 [   0.        0.        1.    ]] 
[[1450.8422    0.     1009.3212]
 [   0.     1413.1678  511.3463]
 [   0.        0.        1.    ]] 
[[2165.2584    0.      951.3176]
 [   0.     2169.1016  499.3   ]
 [   0.        0.        1.    ]] 
[[2165.3993    0.      932.488 ]
 [   0.     2137.3907  524.3095]
 [   0.        0.        1.    ]]


### Save stereo matching frames

In [ ]:
def GenerateImagepoints(Left_Paths:list, Right_Paths:list):
    Left_corners, Right_corners = [], []
    Left_ids, Right_ids = [], []
    Left_Paths_copy, Right_Paths_copy = [], []
    for Lname, Rname in zip(Left_Paths, Right_Paths):

        _, Lcharuco_corners, Lcharuco_ids = findCorners(im_name=Lname)
        _, Rcharuco_corners, Rcharuco_ids = findCorners(im_name=Rname)

        if Lcharuco_corners is not None and Rcharuco_corners is not None:
            Left_corners.append(Lcharuco_corners)
            Left_ids.append(Lcharuco_ids)

            Right_corners.append(Rcharuco_corners)
            Right_ids.append(Rcharuco_ids)

            Left_Paths_copy.append(Lname)
            Right_Paths_copy.append(Rname)

    return Left_corners, Left_ids, Left_Paths_copy, Right_corners, Right_ids, Right_Paths_copy

def getObjectImagePoints(charuco_corners_l, charuco_ids_l, charuco_corners_r, charuco_ids_r):
    common_ids = np.intersect1d(charuco_ids_l, charuco_ids_r, assume_unique=True)
    if len(common_ids) > 0:
        obj_pts = charuco_board.getChessboardCorners()[common_ids.flatten()]
        img_pts_l = charuco_corners_l[np.isin(charuco_ids_l, common_ids)].reshape(-1, 1, 2)
        img_pts_r = charuco_corners_r[np.isin(charuco_ids_r, common_ids)].reshape(-1, 1, 2)
        
        return obj_pts, img_pts_l, img_pts_r, common_ids
    
    return None, None, None, None

In [ ]:
intrinsics_extension = 'mp4'

detector_params = cv2.aruco.DetectorParameters()
video_paths = []

for cam1 in cams:
    path_cam1 = os.path.join(path, 'intrinsics', cam1)

    for cam2 in cams[cams.index(cam1)+1:]:
        path_cam2 = os.path.join(path, 'intrinsics', cam2)
        video_paths = [os.path.join(path_cam1, os.listdir(path_cam1)[0]), os.path.join(path_cam2, os.listdir(path_cam2)[0])]

        output_dir = os.path.join(path, 'stereo', f'{cam1}_{cam2}')
        os.makedirs(output_dir, exist_ok=True)

        valid_frame_count = 0  # This counts frames where both cameras detect the board

        # Open all video captures
        caps = [cv2.VideoCapture(video_path) for video_path in video_paths]

        while True:
            detections = []
            any_failed = False  # Track if any video ended

            for i, cap in enumerate(caps):
                ret, frame = cap.read()
                if not ret:
                    any_failed = True  # Stop processing if any video ends
                    break

                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                corners, ids, _ = cv2.aruco.detectMarkers(gray, aruco_dict, parameters=detector_params)

                if ids is not None and len(ids) >= 10:
                    detections.append((i, frame))

            if any_failed:
                break  # Exit loop if any camera stops

            # Save frames only if both cameras detect the board
            if len(detections) == 2:
                valid_frame_count += 1
                for cam_idx, frame in detections:
                    frame_filename = os.path.join(output_dir, f"c{cam_idx}_frame_{valid_frame_count:03d}.png")
                    cv2.imwrite(frame_filename, frame)

        print(f'{cam1}-{cam2}: Saved {valid_frame_count} valid frames')

c1-c2: Saved 73 valid frames
c1-c3: Saved 0 valid frames
c1-c5: Saved 50 valid frames
c1-c6: Saved 81 valid frames
c1-c7: Saved 60 valid frames
c1-c8: Saved 53 valid frames
c2-c3: Saved 157 valid frames
c2-c5: Saved 58 valid frames
c2-c6: Saved 212 valid frames
c2-c7: Saved 5 valid frames
c2-c8: Saved 33 valid frames
c3-c5: Saved 1 valid frames
c3-c6: Saved 0 valid frames
c3-c7: Saved 0 valid frames
c3-c8: Saved 0 valid frames
c5-c6: Saved 0 valid frames
c5-c7: Saved 93 valid frames
c5-c8: Saved 656 valid frames
c6-c7: Saved 5 valid frames
c6-c8: Saved 0 valid frames
c7-c8: Saved 93 valid frames


In [ ]:
# Create array of all stereo images with 'im_name, img_pts, cam_ timestamp'
stereo_images = {'Name':[], 'Camera':[], 'Corners':[], 'Charuco_Corners':[], 'Ids':[]}
path_stereo = os.path.join(path, 'stereo')

# maximal number of images per camera pair
Nmax = 50

for cam1 in cams:
    for cam2 in cams[cams.index(cam1)+1:]:
        im_saved = 0

        stereopath = os.path.join(path_stereo, f'{cam1}_{cam2}')
        N = len(os.listdir(stereopath))
        
        number_choice = np.linspace(1, int(N//2), int(N//2))
        number_choice = np.round(number_choice).astype(int)

        for i in number_choice:
                
            name1 = f'c0_frame_{i:03d}.png'
            name2 = f'c1_frame_{i:03d}.png'
            
            im1In = name1 in list(os.listdir(stereopath))
            im2In = name2 in list(os.listdir(stereopath))
                    
            if im1In and im2In:
                Lret, Lcorners, Lcharuco_corners, Lcharuco_ids = findCorners(im_name=os.path.join(stereopath, name1))
                Rret, Rcorners, Rcharuco_corners, Rcharuco_ids = findCorners(im_name=os.path.join(stereopath, name2))

                if Lret and Rret:
                    _, _, _, common_ids = getObjectImagePoints(Lcharuco_corners, Lcharuco_ids, Rcharuco_corners, Rcharuco_ids)

                    if len(common_ids) > 1:
                        stereo_images['Name'].append(f'{cam1}_{cam2}_{name1}')
                        stereo_images['Camera'].append(int(cam1[-1]))
                        stereo_images['Corners'].append(Lcorners)
                        stereo_images['Charuco_Corners'].append(Lcharuco_corners)
                        stereo_images['Ids'].append(Lcharuco_ids)

                        stereo_images['Name'].append(f'{cam1}_{cam2}_{name2}')
                        stereo_images['Camera'].append(int(cam2[-1]))
                        stereo_images['Corners'].append(Rcorners)
                        stereo_images['Charuco_Corners'].append(Rcharuco_corners)
                        stereo_images['Ids'].append(Rcharuco_ids)

                        im_saved += 1
            
            if im_saved == Nmax:
                break
        
        print(cam1, cam2, im_saved)
    
with open('stereo_data_trampo.pkl', 'wb') as f:
    pickle.dump(stereo_images, f)

## Stereo initial extrinsic parameters

In [1]:
with open('stereo_data_trampo.pkl', 'rb') as f:
    stereo_images = pickle.load(f)

NameError: name 'pickle' is not defined

In [ ]:
def StereoCalibration(leftparams, rightparams, Left_corners, Left_ids, Right_corners, Right_ids):
    StereoParams = {}
    ret = False
    
    k1 = leftparams['Intrinsic']
    d1 = leftparams['Distortion']
    k2 = rightparams['Intrinsic']
    d2 = rightparams['Distortion']
    
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 1e-5)
    flags = 0
    flags |= cv2.CALIB_FIX_INTRINSIC

    obj_points = []  # 3D world coordinates
    img_points_left = []  # 2D image coordinates (left camera)
    img_points_right = []  # 2D image coordinates (right camera)

    for charuco_corners_l, charuco_ids_l, charuco_corners_r, charuco_ids_r in zip(Left_corners, Left_ids, Right_corners, Right_ids):
        # Find common detected corners (based on Charuco IDs)
        obj_pts, img_pts_l, img_pts_r, common_ids = getObjectImagePoints(charuco_corners_l, charuco_ids_l, charuco_corners_r, charuco_ids_r)
        
        obj_points.append(obj_pts)
        img_points_left.append(img_pts_l)
        img_points_right.append(img_pts_r)

    if len(img_points_left) > 1 and len(img_points_right) > 1:
        (ret, K1, D1, K2, D2, R, t, E, F) = cv2.stereoCalibrate(obj_points, img_points_left, img_points_right, k1, d1, k2, d2, image_size, criteria=criteria, flags=flags)
    
        T = np.vstack((np.hstack((R,t)),np.array([0,0,0,1])))
        
        StereoParams['Transformation'] = T
        StereoParams['Essential'] = E
        StereoParams['Fundamental'] = F
        StereoParams['MeanError'] = ret
        
    return ret, StereoParams

def SaveParameters(camL, camR, Stereo_Params, Left_Params, Right_Params):
    Parameters = Stereo_Params.copy()  # Ensure we're not modifying the original dictionary

    for Lkey in Left_Params.keys():
        name = 'L_'+str(Lkey)
        Parameters[name] = Left_Params[Lkey]
        
    for Rkey in Right_Params.keys():
        name = 'R_'+str(Rkey)
        Parameters[name] = Right_Params[Rkey]

    # Remove imgpoints if they exist in Left_Params and Right_Params
    Parameters = {k: v for k, v in Parameters.items() if k not in ['L_Imgpoints', 'R_Imgpoints']}

    # Save the Parameters dictionary into an npz file
    file = f'{camL}_{camR}_parameters.npz'
    np.savez(file, **Parameters)
    npz = dict(np.load(file))
    np.savez(file, **npz)

    return

In [8]:
path_stereo = os.path.join(path, 'stereo')

for camL in cams[4:]:
    
    Left_Params = {}
    Left_Params['Intrinsic'] = K[cams.index(camL)]
    Left_Params['Distortion'] = D[cams.index(camL)]

    for camR in cams[cams.index(camL)+1:]:
        print(camL, camR)
        stereopath = os.path.join(path_stereo, f'{camL}_{camR}')

        Left_Paths = [os.path.join(stereopath, fname) for fname in list(os.listdir(stereopath)) if fname.split('_')[0] == 'c0']
        Right_Paths = [os.path.join(stereopath, fname) for fname in list(os.listdir(stereopath)) if fname.split('_')[0] == 'c1']
        Right_Params = {}
        
        if len(Left_Paths) >= 1 and len(Right_Paths) >= 1:
            Left_corners, Left_ids, Left_Paths, Right_corners, Right_ids, Right_Paths = GenerateImagepoints(Left_Paths, Right_Paths)
            if len(Left_corners) >= 1 and len(Right_corners) >= 1:

                Right_Params['Intrinsic'] = K[cams.index(camR)]
                Right_Params['Distortion'] = D[cams.index(camR)]
             
                ret, Stereo_Params = StereoCalibration(Left_Params, Right_Params, Left_corners, Left_ids, Right_corners, Right_ids)
                if ret:
                    print('Transformation Matrix:')
                    print(Stereo_Params['Transformation'])

                    SaveParameters(camL, camR, Stereo_Params, Left_Params, Right_Params)
                else:
                    print('Not enough corners', '\n')
            else:
                print('Not enough corners', '\n')
        else:
            print('Not enough images', '\n')

c6 c7
Transformation Matrix:
[[   -0.3904     0.8052     0.4464 -1259.0967]
 [    0.4946    -0.2255     0.8393  2213.5227]
 [    0.7765     0.5485    -0.3102  3367.7477]
 [    0.         0.         0.         1.    ]]
c6 c8
Not enough images 

c7 c8
Transformation Matrix:
[[   0.7521   -0.6315   -0.1883  668.2265]
 [   0.3447    0.6205   -0.7044 2096.9338]
 [   0.5617    0.4649    0.6843  535.2671]
 [   0.        0.        0.        1.    ]]


In [9]:
c1_c2 = np.load('c1_c2_parameters.npz')['Transformation']
c1_c5 = np.load('c1_c5_parameters.npz')['Transformation']
c1_c6 = np.load('c1_c6_parameters.npz')['Transformation']
c1_c8 = np.load('c1_c8_parameters.npz')['Transformation']
c2_c3 = np.load('c2_c3_parameters.npz')['Transformation']
c2_c6 = np.load('c2_c6_parameters.npz')['Transformation']
# c3_c5 = np.load('c3_c5_parameters.npz')['Transformation']
# c5_c8 = np.load('c5_c8_parameters.npz')['Transformation']
c6_c7 = np.load('c6_c7_parameters.npz')['Transformation']
c7_c8 = np.load('c7_c8_parameters.npz')['Transformation']

In [10]:
def var_name(var, scope=globals()):
    return [name for name, value in scope.items() if value is var][0]

def compare_transfo(T12, T23, T13):
    print(f'Original {var_name(T13)}')
    print(T13)
    T13_calc = T12 @ T23
    print(f'Calculated {var_name(T13)}')
    print(T13_calc)
    print('Calculated I')
    print(np.linalg.inv(T13_calc) @ T13, '\n')

    return

In [11]:
compare_transfo(c1_c2, c2_c6, c1_c6)

c6_c8 = c6_c7 @ c7_c8
compare_transfo(c1_c6, c6_c8, c1_c8)

Original c1_c6
[[   -0.9331    -0.2087     0.2928  -412.6195]
 [    0.2952     0.0203     0.9552 -1752.1784]
 [   -0.2053     0.9778     0.0426  2721.545 ]
 [    0.         0.         0.         1.    ]]
Calculated c1_c6
[[  -0.7354    0.5277    0.4251 -515.674 ]
 [   0.6381    0.7504    0.1724  303.7453]
 [  -0.228     0.398    -0.8886 3643.5025]
 [   0.        0.        0.        1.    ]]
Calculated I
[[    0.9214    -0.0565     0.3844 -1177.4193]
 [   -0.3526     0.2943     0.8883 -1855.3634]
 [   -0.1634    -0.954      0.2513   508.6826]
 [    0.         0.         0.         1.    ]] 

Original c1_c8
[[  -0.6748    0.7039    0.2217 -381.9901]
 [  -0.4792   -0.1894   -0.8571 1915.0585]
 [  -0.5613   -0.6846    0.4651 1934.941 ]
 [   0.        0.        0.        1.    ]]
Calculated c1_c8
[[  -0.2034   -0.9631   -0.1761  107.5669]
 [   0.6568   -0.0008   -0.7541 3071.8963]
 [   0.7261   -0.269     0.6328 5309.9665]
 [   0.        0.        0.        1.    ]]
Calculated I
[[   -0.585

## 3D reprojection error

In [13]:
# Function to project 3D points to 2D
def project_points(points_3d, projMat, K, D):
    rvec, _ = cv2.Rodrigues(projMat[0:3,0:3])
    projected_2d, _ = cv2.projectPoints(points_3d, rvec, projMat[0:3,3], K, D)
    return projected_2d.squeeze()

# Compute RMSE
def compute_rmse(original_pts, points_3d, projMat, K, D):
    projected_pts = project_points(points_3d, projMat, K, D)
    error = np.linalg.norm(original_pts - projected_pts, axis=1)  # Euclidean distance per point
    rmse = np.sqrt(np.mean(error**2))  # Compute RMSE
    return rmse

In [15]:
Tcam1 = np.eye((4))
Tcam6 = c1_c6
Tcam5 = c1_c5
Tcam2 = c1_c2
Tcam3 = Tcam2 @ c2_c3
Tcam7 = Tcam6 @ c6_c7
Tcam8 = c1_c8

projMat = [Tcam1, Tcam2, Tcam3, Tcam5, Tcam6, Tcam7, Tcam8]

for mat in projMat:
    print(mat)

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
[[   -0.548     -0.3052     0.7788 -1616.1915]
 [    0.3535     0.7593     0.5463 -1075.6122]
 [   -0.7581     0.5747    -0.3082  2708.2644]
 [    0.         0.         0.         1.    ]]
[[   0.1536    0.3643    0.9185  599.4048]
 [  -0.776    -0.5311    0.3403  655.6455]
 [   0.6118   -0.765     0.2011 2491.6402]
 [   0.        0.        0.        1.    ]]
[[   -0.9003    -0.2442     0.3603 -1511.0614]
 [   -0.4162     0.2407    -0.8768  2200.7504]
 [    0.1274    -0.9394    -0.3183  3113.172 ]
 [    0.         0.         0.         1.    ]]
[[   -0.9331    -0.2087     0.2928  -412.6195]
 [    0.2952     0.0203     0.9552 -1752.1784]
 [   -0.2053     0.9778     0.0426  2721.545 ]
 [    0.         0.         0.         1.    ]]
[[   0.4884   -0.5437   -0.6825 1286.5337]
 [   0.6365    0.757    -0.1475 1138.061 ]
 [   0.5969   -0.3624    0.7158 5287.8902]
 [   0.        0.        0.        1.    ]]
[[  -0.6748    0.7039    0

## PSO for extrinsic parameters (with Bundle adjustment)

In [27]:
# Constants
num_cameras = 7
num_params_per_cam = 6
total_params = num_cameras * num_params_per_cam
n_particles = 60  # Number of particles
options = {'c1': 1.5, 'c2': 1.5, 'w': 0.7} #{'c1': 2.05, 'c2': 2.05, 'w': 0.729} # PSO Hyperparameters

projMat = np.load('Extrinsics_optimized_3.npz')['arr_0']

init_params = []
for mat in projMat:
    rvec, _ = cv2.Rodrigues(mat[0:3,0:3])
    rvec = rvec.squeeze()
    params = [rvec[0], rvec[1], rvec[2], mat[0,3], mat[1,3], mat[2,3]]
    init_params.extend(params)

init_params = np.array(init_params, dtype=np.float64)
init_pos = np.tile(init_params, (n_particles,1))
print(init_params.shape)
print(init_pos.shape)

# Define parameter bounds
lower_bounds = [-2*np.pi, -2*np.pi, -2*np.pi, -5000, -5000, -5000]
upper_bounds = [2*np.pi, 2*np.pi, 2*np.pi, 5000, 5000, 6000]

param_bounds = (np.tile(lower_bounds, num_cameras), np.tile(upper_bounds, num_cameras))
print(param_bounds[0].shape)

print(np.all(param_bounds[0] <= init_pos[0]))
print(np.all(init_pos[0] <= param_bounds[1]))

(42,)
(60, 42)
(42,)
True
True


In [28]:
def fun(params):
    errors = np.empty((n_particles,))
    params = np.array(params)

    for n in range(n_particles):
        projMat = np.empty((num_cameras+1, 3, 4))
        projMat[0] = np.hstack((np.eye((3)), np.zeros((3,1))))

        for cam_idx in range(num_cameras):
            cam_params = params[n][cam_idx * num_params_per_cam : (cam_idx + 1) * num_params_per_cam]
            r1, r2, r3, t1, t2, t3 = cam_params
            rvec = np.array([r1, r2, r3])
            R, _ = cv2.Rodrigues(rvec)
            t = np.array([t1, t2, t3]).reshape((3,1))
            projMat[cam_idx+1] = np.hstack((R, t))  #[cam_idx+1]

        RMSE = []
        # Loop on stereo images checkberboard points
        for i in range(0, len(stereo_images['Camera']) - 1, 2):
            j = i+1 # stereo image is the next one

            pts1_im = stereo_images['Charuco_Corners'][i].squeeze()
            pts2_im = stereo_images['Charuco_Corners'][j].squeeze()

            Lids = stereo_images['Ids'][i].squeeze()
            Rids = stereo_images['Ids'][j].squeeze()

            c1 = cams.index(f'c{stereo_images['Camera'][i]}')
            c2 = cams.index(f'c{stereo_images['Camera'][j]}')

            undist_pts1 = cv2.undistortPoints(pts1_im, K[c1], D[c1]).reshape(-1, 2)  # Shape (2, N)
            undist_pts2 = cv2.undistortPoints(pts2_im, K[c2], D[c2]).reshape(-1, 2)  # Shape (2, N)

            obj_pts, img_pts_l, img_pts_r, common_ids = getObjectImagePoints(undist_pts1, Lids, undist_pts2, Rids)

            ids_to_keepL = [list(Lids).index(t) for t in common_ids]
            ids_to_keepR = [list(Rids).index(t) for t in common_ids]

            img_pts_l, img_pts_r = img_pts_l.squeeze(), img_pts_r.squeeze()

            # Perform triangulation
            pts_4d = cv2.triangulatePoints(projMat[c1], projMat[c2], img_pts_l.T, img_pts_r.T)
            points_3d = pts_4d[:3, :] / pts_4d[3, :]  # Shape (3, N)
            points_3d = points_3d.T  # Shape (N, 3)

            # Compute RMSE for both cameras
            rmse1 = compute_rmse(pts1_im[ids_to_keepL], points_3d, projMat[c1], K[c1], D[c1])
            rmse2 = compute_rmse(pts2_im[ids_to_keepR], points_3d, projMat[c2], K[c2], D[c2])

            RMSE.append(rmse1)
            RMSE.append(rmse2)

        errors[n] = np.mean(RMSE) #+ np.max(RMSE)   # OPTIONAL: add max error to cost function
    
    return errors


In [29]:
# Run PSO to optimize calibration parameters
optimizer = ps.single.GlobalBestPSO(n_particles=n_particles, dimensions=total_params, options=options, bounds=param_bounds, init_pos=init_pos)
best_error, best_params = optimizer.optimize(fun, iters=2000)
print(f"Best Reprojection Error: {best_error} pixels")

# Extract optimized parameters for each camera
optimized_params = np.split(best_params, num_cameras)

plot_cost_history(cost_history=optimizer.cost_history)
#plt.savefig('Figure_1_pso_cost_3.png')
plt.show()

2025-03-28 18:54:00,332 - pyswarms.single.global_best - INFO - Optimize for 2000 iters with {'c1': 1.5, 'c2': 1.5, 'w': 0.7}
pyswarms.single.global_best: 100%|██████████|2000/2000, best_cost=8.41
2025-03-28 20:10:49,739 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 8.41150188446045, best pos: [    0.1342     0.4142    -0.6978  1452.0612  -595.7068  3208.5055
    -0.4775     1.0714    -1.4878  -425.9488 -1267.6664  5593.3008
    -0.1449     0.5425    -1.8993 -4172.4578  4938.328   4958.2298
     0.7131     2.9304    -0.6542 -2945.081   1637.1761  3869.3454
     0.9303    -2.8937    -0.0815   490.6938 -1614.5824 -1634.6394
    -0.1349     0.0269    -2.1777  -520.9657  3531.7928  4463.8015
     0.9667    -3.2931    -3.489   1165.8387  -600.8255 -3669.6917]


Best Reprojection Error: 8.41150188446045 pixels


In [30]:
extrinsics = []
extrinsics.append(np.hstack((np.eye((3)), np.zeros((3,1)))))

for cam_idx in range(num_cameras):
    r1, r2, r3, t1, t2, t3 = optimized_params[cam_idx]
    rvec = np.array([r1,r2,r3], dtype=np.float64)
    R, _ = cv2.Rodrigues(rvec)
    t = np.array([t1,t2,t3], dtype=np.float64).reshape((3,1))

    optimized_camera_matrix = np.hstack((R,t))
    extrinsics.append(optimized_camera_matrix)

for cam_idx, cam_mat in enumerate(extrinsics):
    print(f"Camera {cam_idx+1} Optimized Camera Matrix:\n", cam_mat)

np.savez('Extrinsics_optimized_4.npz', extrinsics[1:])

Camera 1 Optimized Camera Matrix:
 [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]
Camera 2 Optimized Camera Matrix:
 [[   0.6889    0.648     0.3248 1452.0612]
 [  -0.5955    0.7614   -0.2561 -595.7068]
 [  -0.4133   -0.017     0.9104 3208.5055]]
Camera 3 Optimized Camera Matrix:
 [[   -0.2345     0.5566     0.797   -425.9488]
 [   -0.9324     0.1034    -0.3465 -1267.6664]
 [   -0.2752    -0.8243     0.4947  5593.3008]]
Camera 4 Optimized Camera Matrix:
 [[   -0.391      0.8515     0.3493 -4172.4578]
 [   -0.9076    -0.2935    -0.3002  4938.328 ]
 [   -0.1531    -0.4344     0.8876  4958.2298]]
Camera 5 Optimized Camera Matrix:
 [[   -0.8917     0.4503    -0.0452 -2945.081 ]
 [    0.4268     0.8035    -0.4151  1637.1761]
 [   -0.1506    -0.3894    -0.9087  3869.3454]]
Camera 6 Optimized Camera Matrix:
 [[   -0.8082    -0.5781    -0.1122   490.6938]
 [   -0.5835     0.8118     0.0201 -1614.5824]
 [    0.0795     0.0817    -0.9935 -1634.6394]]
Camera 7 Optimized Camera Matrix:
 [[  -0.5679

## Cumulative error on matrices multiplication (Total transfo = I ?) 

In [31]:
T = []

for mat in extrinsics:
    M = np.vstack((mat, np.array([0,0,0,1])))
    T.append(M)

T_next = []
for i in range(len(T) - 1):
    T_next.append(np.linalg.inv(T[i]) @ T[i+1])

T5_calc = T_next[0] @ T_next[1] @ T_next[2] @ T_next[3]

print('T5 calc: \n', T5_calc)
print('T5: \n', T[4])

I_calc = T5_calc @ np.linalg.inv(T[4])
print('Cumulative error: \n', np.abs(I_calc - np.eye(4)))

T5 calc: 
 [[   -0.8917     0.4503    -0.0452 -2945.081 ]
 [    0.4268     0.8035    -0.4151  1637.1761]
 [   -0.1506    -0.3894    -0.9087  3869.3454]
 [    0.         0.         0.         1.    ]]
T5: 
 [[   -0.8917     0.4503    -0.0452 -2945.081 ]
 [    0.4268     0.8035    -0.4151  1637.1761]
 [   -0.1506    -0.3894    -0.9087  3869.3454]
 [    0.         0.         0.         1.    ]]
Cumulative error: 
 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


## 2D visualization of original and projected points onto images

In [25]:
def draw_circles(img, pts_2d_original, pts_2d_projected, text=True):

    pts_2d_original = pts_2d_original.reshape(-1,2)
    pts_2d_projected = pts_2d_projected.reshape(-1,2)
    
    for p in range(len(pts_2d_original)):
        x_orig, y_orig = map(int, pts_2d_original[p])  # Original 2D point
        cv2.circle(img, (x_orig, y_orig), 8, (255, 0, 0), -1)

        if pts_2d_projected is not None:
            x_proj, y_proj = map(int, pts_2d_projected[p])  # Reprojected 3D point
            cv2.circle(img, (x_proj, y_proj), 8, (0, 255, 0), -1)
            cv2.line(img, (x_orig, y_orig), (x_proj, y_proj), (0, 255, 255), 2)

        if text:
            # Add labels
            cv2.putText(img, str(p + 1), (x_orig + 10, y_orig), cv2.FONT_HERSHEY_SIMPLEX, 
                        0.6, (255, 0, 0), 2, cv2.LINE_AA)
            cv2.putText(img, str(p + 1), (x_proj + 10, y_proj), cv2.FONT_HERSHEY_SIMPLEX, 
                        0.6, (0, 255, 0), 2, cv2.LINE_AA)
        
    return img

In [32]:
dir_images = r'C:\Users\LEA\Desktop\Poly\Trampo\video_test\stereo'

projMat = extrinsics

RMSE = []
rmse_per_cam = {'c1-c2':[], 'c1-c3':[], 'c1-c5':[], 'c1-c6':[], 'c1-c7':[], 'c1-c8':[],
                'c2-c3':[], 'c2-c5':[], 'c2-c6':[], 'c2-c7':[], 'c2-c8':[],
                'c3-c5':[], 'c3-c6':[], 'c3-c7':[], 'c3-c8':[],
                'c5-c6':[], 'c5-c7':[], 'c5-c8':[], 'c6-c7':[], 'c6-c8':[], 'c7-c8':[]}

fig, axs = plt.subplots(6, 6, figsize=(20,14))
axs = axs.flat
k = 0

for i in range(0, len(stereo_images['Camera']) - 1, 2):
    j = i+1 # stereo image is the next one

    pts1_im = stereo_images['Charuco_Corners'][i].squeeze()

    pts2_im = stereo_images['Charuco_Corners'][j].squeeze()

    Lids = stereo_images['Ids'][i].squeeze()
    Rids = stereo_images['Ids'][j].squeeze()

    c1 = cams.index(f'c{stereo_images['Camera'][i]}')
    c2 = cams.index(f'c{stereo_images['Camera'][j]}')
    
    undist_pts1 = cv2.undistortPoints(pts1_im, K[c1], D[c1]).reshape(-1, 2)  # Shape (2, N)
    undist_pts2 = cv2.undistortPoints(pts2_im, K[c2], D[c2]).reshape(-1, 2)  # Shape (2, N)

    obj_pts, img_pts_l, img_pts_r, common_ids = getObjectImagePoints(undist_pts1, Lids, undist_pts2, Rids)

    ids_to_keepL = [list(Lids).index(t) for t in common_ids]
    ids_to_keepR = [list(Rids).index(t) for t in common_ids]

    img_pts_l, img_pts_r = img_pts_l.squeeze(), img_pts_r.squeeze()

    # Perform triangulation
    pts_4d = cv2.triangulatePoints(projMat[c1], projMat[c2], img_pts_l.T, img_pts_r.T)
    points_3d = pts_4d[:3, :] / pts_4d[3, :]  # Shape (3, N)
    points_3d = points_3d.T  # Shape (N, 3)

    img_path1 = os.path.join(dir_images, stereo_images['Name'][i][0:5], stereo_images['Name'][i][6:])
    img_path2 = os.path.join(dir_images, stereo_images['Name'][j][0:5], stereo_images['Name'][j][6:])

    # Visualize original and reprojected points (image 1)
    img = cv2.imread(img_path1)

    pts_2d_projected, _ = cv2.projectPoints(points_3d, projMat[c1][0:3,0:3], projMat[c1][0:3,3], K[c1], D[c1])
    pts_2d_projected = pts_2d_projected.squeeze()
    rmse1 = compute_rmse(pts1_im[ids_to_keepL], points_3d, projMat[c1], K[c1], D[c1])

    if i % 8 == 0:
        img = draw_circles(img, pts1_im[ids_to_keepL], pts_2d_projected)
        axs[k].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        axs[k].axis('off')
        axs[k].set_title(f'{stereo_images['Name'][i]} RMSE = {rmse1:.2f}', fontsize=8)

    # Visualize original and reprojected points (image 2)
    img = cv2.imread(img_path2)
    
    pts_2d_projected, _ = cv2.projectPoints(points_3d, projMat[c2][0:3,0:3], projMat[c2][0:3,3], K[c2], D[c2])
    pts_2d_projected = pts_2d_projected.squeeze()
    rmse2 = compute_rmse(pts2_im[ids_to_keepR], points_3d, projMat[c2], K[c2], D[c2])

    if i % 8 == 0:
        img = draw_circles(img, pts2_im[ids_to_keepR], pts_2d_projected)
        axs[k+1].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        axs[k+1].axis('off')
        axs[k+1].set_title(f'{stereo_images['Name'][j]} RMSE = {rmse2:.2f}', fontsize=8)
        k += 2

    rmse_per_cam[f'{cams[c1]}-{cams[c2]}'] += [rmse1, rmse2]

fig.suptitle("Original 2D Points (Blue) vs. Projected 3D Points (Green)")
fig.tight_layout()
plt.savefig('Figure_reprojection_4.png')
plt.show()
plt.close()

for key, vals in rmse_per_cam.items():
    print(key, np.mean(vals))

c1-c2 5.35145
c1-c3 nan
c1-c5 6.7054524
c1-c6 9.101836
c1-c7 nan
c1-c8 12.525467
c2-c3 6.508746
c2-c5 nan
c2-c6 35.759777
c2-c7 nan
c2-c8 nan
c3-c5 2.2967505
c3-c6 nan
c3-c7 nan
c3-c8 nan
c5-c6 nan
c5-c7 nan
c5-c8 0.8153839
c6-c7 3.1119726
c6-c8 nan
c7-c8 5.6156445


## 3D visualization of cameras in space

In [33]:
def plot_frame(ax, T, label="Frame", length=100.0):
    """Plots a 3D reference frame with arrows for X, Y, and Z axes."""
    origin = T[:3, 3]  # Extract translation (position)
    R = T[:3, :3]      # Extract rotation matrix (orientation)

    # Define the x, y, z unit vectors from the rotation matrix
    x_axis = R[:, 0] * length  # X-direction
    y_axis = R[:, 1] * length  # Y-direction
    z_axis = R[:, 2] * length  # Z-direction

    # Plot arrows for the axes
    ax.quiver(*origin, *x_axis, color='r', arrow_length_ratio=0.3)  # X-axis (Red)
    ax.quiver(*origin, *y_axis, color='g', arrow_length_ratio=0.3)  # Y-axis (Green)
    ax.quiver(*origin, *z_axis, color='b', arrow_length_ratio=0.3)  # Z-axis (Blue)

    # Label the frame
    ax.text(*origin, label, fontsize=12, color='black')

    return

In [34]:
# Plot frames
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

for i, T in enumerate(extrinsics):
    T = np.vstack((T, np.array([0,0,0,1])))
    plot_frame(ax, T, label=f'Camera {i+1}')

# Labels
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
plt.title("3D Reference Frames")
plt.show()